In [24]:
import torchvision.models as models
import pydicom
from torch.utils.data import Dataset,DataLoader
import os
import pylab
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch
import cv2
from openpyxl import Workbook

In [44]:
'''
0->硬膜外出血 epidural: 223
1->健康 healthy: 977
2->腦實質性出血 intraparenchymal: 807
3->腦室內出血 intraventricular: 712
4->蛛網膜下腔出血 subarachnoid: 717
5->硬膜下出血 subdural: 794
'''

class_dict={
    0:'epidural',
    1:'healthy',
    2:'intraparenchymal',
    3:'intraventricular',
    4:'subarachnoid',
    5:'subdural'
}

In [2]:
device = torch.cuda.current_device()
print('device:',device)

device: 0


In [46]:
class ICHDataset(Dataset):
    def __init__(self,root,train=True):
        self.train=train
        self.labels=[]
        self.filename_list=[]
        
        if train:
            all_dirs=os.listdir(root)

            label=0
            for dir in all_dirs:
                path=os.path.join(root,dir)
                all_files=os.listdir(path)
                for i,file in enumerate(all_files):
                    self.filename_list.append(os.path.join(path,file))
                    self.labels.append(label)
                label+=1
    
        if not train:
            all_files=os.listdir(root)
            for file in all_files:
                self.filename_list.append(os.path.join(root,file))
            
            
    def __getitem__(self,idx):
        filename=self.filename_list[idx]
        pixel_arr=pydicom.read_file(filename).pixel_array
        pixel_arr=cv2.resize(pixel_arr,(512,512))
        pixel_arr=np.float32(pixel_arr)
        pixel_arr=cv2.cvtColor(pixel_arr, cv2.COLOR_GRAY2BGR)
        transform = transforms.Compose([            
            transforms.ToTensor()
        ])
        data=transform(pixel_arr)
        if self.train:
            target=self.labels[idx]
            return data,target
        else:
            return data
        
    def __len__(self):
        return len(self.filename_list)
                


In [47]:
epochs=23
lr=0.01
batch_size=16
criterion=nn.CrossEntropyLoss()
num_classes=6

In [48]:
trainset=ICHDataset('TrainingData')
dataloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
print(trainset[0][0])

tensor([[[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]]

In [50]:
resnet18 = models.resnet18()
resnet18.fc=nn.Linear(512,num_classes)
resnet18=resnet18.to(device)

resnet18_pretrain=resnet18 = models.resnet18(pretrained=True)
resnet18_pretrain.fc=nn.Linear(512,num_classes)
resnet18_pretrain=resnet18_pretrain.to(device)

optimizer=optim.SGD(resnet18.parameters(),lr=lr,momentum=0.9,weight_decay=0.0005)

In [12]:
#Train
def train(model,dataloader):
    for epoch in range(epochs):
        correct=0.0
        running_loss=0.0 
        for times,data in enumerate(dataloader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs,labels=inputs.to(device),labels.to(device)
            outputs=model(inputs)
            _, predict = torch.max(outputs.data, 1)
            correct+=(predict==labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print('[%d/%d] [%d/%d] loss: %.3f' % (epoch+1, epochs, times, len(trainset)/batch_size, running_loss))
        
        accuracy=correct/len(trainset)*100
        print('Train accuracy: %.3f%%' % accuracy)

train(resnet18,dataloader)

[1/23] [0/375] loss: 1.621
[1/23] [1/375] loss: 3.641
[1/23] [2/375] loss: 5.321
[1/23] [3/375] loss: 7.205
[1/23] [4/375] loss: 8.853
[1/23] [5/375] loss: 10.495
[1/23] [6/375] loss: 12.065
[1/23] [7/375] loss: 13.591
[1/23] [8/375] loss: 15.220
[1/23] [9/375] loss: 16.881
[1/23] [10/375] loss: 18.646
[1/23] [11/375] loss: 20.614
[1/23] [12/375] loss: 22.702
[1/23] [13/375] loss: 24.579
[1/23] [14/375] loss: 26.187
[1/23] [15/375] loss: 28.349
[1/23] [16/375] loss: 30.432
[1/23] [17/375] loss: 32.159
[1/23] [18/375] loss: 33.871
[1/23] [19/375] loss: 35.503
[1/23] [20/375] loss: 37.258
[1/23] [21/375] loss: 39.199
[1/23] [22/375] loss: 40.951
[1/23] [23/375] loss: 42.679
[1/23] [24/375] loss: 44.576
[1/23] [25/375] loss: 46.197
[1/23] [26/375] loss: 47.777
[1/23] [27/375] loss: 49.574
[1/23] [28/375] loss: 51.460
[1/23] [29/375] loss: 53.170
[1/23] [30/375] loss: 54.872
[1/23] [31/375] loss: 56.591
[1/23] [32/375] loss: 58.490
[1/23] [33/375] loss: 60.084
[1/23] [34/375] loss: 61.793


[1/23] [271/375] loss: 467.657
[1/23] [272/375] loss: 469.206
[1/23] [273/375] loss: 470.988
[1/23] [274/375] loss: 472.776
[1/23] [275/375] loss: 474.357
[1/23] [276/375] loss: 475.875
[1/23] [277/375] loss: 477.581
[1/23] [278/375] loss: 479.198
[1/23] [279/375] loss: 480.928
[1/23] [280/375] loss: 482.986
[1/23] [281/375] loss: 484.831
[1/23] [282/375] loss: 486.507
[1/23] [283/375] loss: 488.017
[1/23] [284/375] loss: 489.681
[1/23] [285/375] loss: 491.475
[1/23] [286/375] loss: 493.186
[1/23] [287/375] loss: 494.654
[1/23] [288/375] loss: 496.307
[1/23] [289/375] loss: 498.031
[1/23] [290/375] loss: 499.682
[1/23] [291/375] loss: 501.312
[1/23] [292/375] loss: 502.907
[1/23] [293/375] loss: 504.607
[1/23] [294/375] loss: 506.269
[1/23] [295/375] loss: 508.334
[1/23] [296/375] loss: 509.983
[1/23] [297/375] loss: 511.565
[1/23] [298/375] loss: 513.131
[1/23] [299/375] loss: 514.789
[1/23] [300/375] loss: 516.426
[1/23] [301/375] loss: 518.199
[1/23] [302/375] loss: 519.775
[1/23] [

[2/23] [166/375] loss: 282.581
[2/23] [167/375] loss: 284.413
[2/23] [168/375] loss: 285.850
[2/23] [169/375] loss: 287.470
[2/23] [170/375] loss: 288.947
[2/23] [171/375] loss: 290.618
[2/23] [172/375] loss: 292.301
[2/23] [173/375] loss: 293.835
[2/23] [174/375] loss: 295.612
[2/23] [175/375] loss: 297.078
[2/23] [176/375] loss: 298.764
[2/23] [177/375] loss: 300.205
[2/23] [178/375] loss: 301.747
[2/23] [179/375] loss: 303.403
[2/23] [180/375] loss: 304.868
[2/23] [181/375] loss: 306.492
[2/23] [182/375] loss: 307.876
[2/23] [183/375] loss: 309.475
[2/23] [184/375] loss: 311.224
[2/23] [185/375] loss: 312.894
[2/23] [186/375] loss: 314.612
[2/23] [187/375] loss: 316.296
[2/23] [188/375] loss: 317.605
[2/23] [189/375] loss: 319.199
[2/23] [190/375] loss: 320.695
[2/23] [191/375] loss: 322.249
[2/23] [192/375] loss: 323.794
[2/23] [193/375] loss: 325.475
[2/23] [194/375] loss: 327.082
[2/23] [195/375] loss: 328.482
[2/23] [196/375] loss: 330.128
[2/23] [197/375] loss: 331.822
[2/23] [

[3/23] [59/375] loss: 98.998
[3/23] [60/375] loss: 100.692
[3/23] [61/375] loss: 102.075
[3/23] [62/375] loss: 103.670
[3/23] [63/375] loss: 105.139
[3/23] [64/375] loss: 107.000
[3/23] [65/375] loss: 108.648
[3/23] [66/375] loss: 110.150
[3/23] [67/375] loss: 111.759
[3/23] [68/375] loss: 113.277
[3/23] [69/375] loss: 115.062
[3/23] [70/375] loss: 116.759
[3/23] [71/375] loss: 118.065
[3/23] [72/375] loss: 119.647
[3/23] [73/375] loss: 121.406
[3/23] [74/375] loss: 122.830
[3/23] [75/375] loss: 124.328
[3/23] [76/375] loss: 126.360
[3/23] [77/375] loss: 127.929
[3/23] [78/375] loss: 129.376
[3/23] [79/375] loss: 130.803
[3/23] [80/375] loss: 132.470
[3/23] [81/375] loss: 134.249
[3/23] [82/375] loss: 135.959
[3/23] [83/375] loss: 137.893
[3/23] [84/375] loss: 139.457
[3/23] [85/375] loss: 141.141
[3/23] [86/375] loss: 142.640
[3/23] [87/375] loss: 144.369
[3/23] [88/375] loss: 145.963
[3/23] [89/375] loss: 147.433
[3/23] [90/375] loss: 149.077
[3/23] [91/375] loss: 150.685
[3/23] [92/

[3/23] [326/375] loss: 534.449
[3/23] [327/375] loss: 536.313
[3/23] [328/375] loss: 538.127
[3/23] [329/375] loss: 539.883
[3/23] [330/375] loss: 541.474
[3/23] [331/375] loss: 543.239
[3/23] [332/375] loss: 544.677
[3/23] [333/375] loss: 546.468
[3/23] [334/375] loss: 548.096
[3/23] [335/375] loss: 549.625
[3/23] [336/375] loss: 551.304
[3/23] [337/375] loss: 552.900
[3/23] [338/375] loss: 554.730
[3/23] [339/375] loss: 556.068
[3/23] [340/375] loss: 557.671
[3/23] [341/375] loss: 559.206
[3/23] [342/375] loss: 561.014
[3/23] [343/375] loss: 563.018
[3/23] [344/375] loss: 564.552
[3/23] [345/375] loss: 565.939
[3/23] [346/375] loss: 567.358
[3/23] [347/375] loss: 569.016
[3/23] [348/375] loss: 570.647
[3/23] [349/375] loss: 572.104
[3/23] [350/375] loss: 573.640
[3/23] [351/375] loss: 575.349
[3/23] [352/375] loss: 576.958
[3/23] [353/375] loss: 578.310
[3/23] [354/375] loss: 579.916
[3/23] [355/375] loss: 581.615
[3/23] [356/375] loss: 583.089
[3/23] [357/375] loss: 584.905
[3/23] [

...

[20/23] [277/375] loss: 333.421
[20/23] [278/375] loss: 334.881
[20/23] [279/375] loss: 336.267
[20/23] [280/375] loss: 337.870
[20/23] [281/375] loss: 338.989
[20/23] [282/375] loss: 340.197
[20/23] [283/375] loss: 341.299
[20/23] [284/375] loss: 342.390
[20/23] [285/375] loss: 343.864
[20/23] [286/375] loss: 345.180
[20/23] [287/375] loss: 346.104
[20/23] [288/375] loss: 347.358
[20/23] [289/375] loss: 348.782
[20/23] [290/375] loss: 350.559
[20/23] [291/375] loss: 351.495
[20/23] [292/375] loss: 352.814
[20/23] [293/375] loss: 354.065
[20/23] [294/375] loss: 355.338
[20/23] [295/375] loss: 356.640
[20/23] [296/375] loss: 357.628
[20/23] [297/375] loss: 358.540
[20/23] [298/375] loss: 359.678
[20/23] [299/375] loss: 361.155
[20/23] [300/375] loss: 362.241
[20/23] [301/375] loss: 363.025
[20/23] [302/375] loss: 364.167
[20/23] [303/375] loss: 364.983
[20/23] [304/375] loss: 366.228
[20/23] [305/375] loss: 367.165
[20/23] [306/375] loss: 368.446
[20/23] [307/375] loss: 369.792
[20/23] 

[21/23] [165/375] loss: 180.593
[21/23] [166/375] loss: 181.498
[21/23] [167/375] loss: 182.707
[21/23] [168/375] loss: 183.875
[21/23] [169/375] loss: 185.157
[21/23] [170/375] loss: 186.434
[21/23] [171/375] loss: 187.931
[21/23] [172/375] loss: 188.943
[21/23] [173/375] loss: 189.748
[21/23] [174/375] loss: 190.826
[21/23] [175/375] loss: 191.748
[21/23] [176/375] loss: 193.025
[21/23] [177/375] loss: 194.125
[21/23] [178/375] loss: 195.252
[21/23] [179/375] loss: 196.349
[21/23] [180/375] loss: 197.455
[21/23] [181/375] loss: 198.637
[21/23] [182/375] loss: 199.435
[21/23] [183/375] loss: 200.581
[21/23] [184/375] loss: 201.458
[21/23] [185/375] loss: 202.183
[21/23] [186/375] loss: 203.182
[21/23] [187/375] loss: 203.881
[21/23] [188/375] loss: 205.179
[21/23] [189/375] loss: 206.701
[21/23] [190/375] loss: 207.785
[21/23] [191/375] loss: 208.928
[21/23] [192/375] loss: 210.316
[21/23] [193/375] loss: 211.285
[21/23] [194/375] loss: 212.303
[21/23] [195/375] loss: 213.312
[21/23] 

[22/23] [50/375] loss: 50.603
[22/23] [51/375] loss: 51.677
[22/23] [52/375] loss: 53.004
[22/23] [53/375] loss: 54.014
[22/23] [54/375] loss: 55.293
[22/23] [55/375] loss: 56.259
[22/23] [56/375] loss: 57.292
[22/23] [57/375] loss: 58.179
[22/23] [58/375] loss: 59.119
[22/23] [59/375] loss: 60.091
[22/23] [60/375] loss: 60.555
[22/23] [61/375] loss: 61.847
[22/23] [62/375] loss: 62.598
[22/23] [63/375] loss: 63.784
[22/23] [64/375] loss: 64.828
[22/23] [65/375] loss: 65.336
[22/23] [66/375] loss: 66.450
[22/23] [67/375] loss: 67.619
[22/23] [68/375] loss: 69.021
[22/23] [69/375] loss: 70.022
[22/23] [70/375] loss: 71.032
[22/23] [71/375] loss: 72.311
[22/23] [72/375] loss: 74.102
[22/23] [73/375] loss: 75.347
[22/23] [74/375] loss: 76.393
[22/23] [75/375] loss: 77.132
[22/23] [76/375] loss: 78.170
[22/23] [77/375] loss: 79.418
[22/23] [78/375] loss: 80.365
[22/23] [79/375] loss: 81.348
[22/23] [80/375] loss: 82.437
[22/23] [81/375] loss: 83.338
[22/23] [82/375] loss: 84.501
[22/23] [8

[22/23] [310/375] loss: 322.793
[22/23] [311/375] loss: 323.617
[22/23] [312/375] loss: 325.097
[22/23] [313/375] loss: 326.242
[22/23] [314/375] loss: 327.272
[22/23] [315/375] loss: 328.350
[22/23] [316/375] loss: 329.135
[22/23] [317/375] loss: 330.004
[22/23] [318/375] loss: 330.712
[22/23] [319/375] loss: 331.842
[22/23] [320/375] loss: 332.756
[22/23] [321/375] loss: 334.120
[22/23] [322/375] loss: 335.223
[22/23] [323/375] loss: 336.351
[22/23] [324/375] loss: 337.578
[22/23] [325/375] loss: 338.384
[22/23] [326/375] loss: 339.565
[22/23] [327/375] loss: 340.740
[22/23] [328/375] loss: 341.864
[22/23] [329/375] loss: 342.817
[22/23] [330/375] loss: 343.998
[22/23] [331/375] loss: 345.059
[22/23] [332/375] loss: 346.355
[22/23] [333/375] loss: 347.643
[22/23] [334/375] loss: 349.101
[22/23] [335/375] loss: 350.097
[22/23] [336/375] loss: 351.139
[22/23] [337/375] loss: 352.581
[22/23] [338/375] loss: 353.871
[22/23] [339/375] loss: 355.071
[22/23] [340/375] loss: 356.303
[22/23] 

[23/23] [198/375] loss: 186.456
[23/23] [199/375] loss: 187.587
[23/23] [200/375] loss: 188.622
[23/23] [201/375] loss: 189.470
[23/23] [202/375] loss: 190.028
[23/23] [203/375] loss: 190.802
[23/23] [204/375] loss: 191.728
[23/23] [205/375] loss: 192.963
[23/23] [206/375] loss: 193.918
[23/23] [207/375] loss: 194.584
[23/23] [208/375] loss: 195.189
[23/23] [209/375] loss: 196.297
[23/23] [210/375] loss: 197.275
[23/23] [211/375] loss: 198.396
[23/23] [212/375] loss: 199.532
[23/23] [213/375] loss: 200.324
[23/23] [214/375] loss: 201.239
[23/23] [215/375] loss: 202.321
[23/23] [216/375] loss: 203.199
[23/23] [217/375] loss: 204.178
[23/23] [218/375] loss: 205.209
[23/23] [219/375] loss: 206.107
[23/23] [220/375] loss: 207.564
[23/23] [221/375] loss: 209.030
[23/23] [222/375] loss: 209.552
[23/23] [223/375] loss: 210.436
[23/23] [224/375] loss: 211.616
[23/23] [225/375] loss: 212.680
[23/23] [226/375] loss: 213.671
[23/23] [227/375] loss: 214.479
[23/23] [228/375] loss: 215.316
[23/23] 

In [51]:
train(resnet18_pretrain,dataloader)

[1/23] [0/375] loss: 2.254
[1/23] [1/375] loss: 4.054
[1/23] [2/375] loss: 5.979
[1/23] [3/375] loss: 7.817
[1/23] [4/375] loss: 9.674
[1/23] [5/375] loss: 11.391
[1/23] [6/375] loss: 14.183
[1/23] [7/375] loss: 15.888
[1/23] [8/375] loss: 18.804
[1/23] [9/375] loss: 20.809
[1/23] [10/375] loss: 23.003
[1/23] [11/375] loss: 25.842
[1/23] [12/375] loss: 28.008
[1/23] [13/375] loss: 30.696
[1/23] [14/375] loss: 33.105
[1/23] [15/375] loss: 35.157
[1/23] [16/375] loss: 37.323
[1/23] [17/375] loss: 39.464
[1/23] [18/375] loss: 43.589
[1/23] [19/375] loss: 46.383
[1/23] [20/375] loss: 49.814
[1/23] [21/375] loss: 52.289
[1/23] [22/375] loss: 54.940
[1/23] [23/375] loss: 57.839
[1/23] [24/375] loss: 60.688
[1/23] [25/375] loss: 62.944
[1/23] [26/375] loss: 66.557
[1/23] [27/375] loss: 68.608
[1/23] [28/375] loss: 70.464
[1/23] [29/375] loss: 72.210
[1/23] [30/375] loss: 75.413
[1/23] [31/375] loss: 77.589
[1/23] [32/375] loss: 79.685
[1/23] [33/375] loss: 81.644
[1/23] [34/375] loss: 84.135


[1/23] [270/375] loss: 515.299
[1/23] [271/375] loss: 517.083
[1/23] [272/375] loss: 518.576
[1/23] [273/375] loss: 520.282
[1/23] [274/375] loss: 522.001
[1/23] [275/375] loss: 523.532
[1/23] [276/375] loss: 525.196
[1/23] [277/375] loss: 526.926
[1/23] [278/375] loss: 528.912
[1/23] [279/375] loss: 530.780
[1/23] [280/375] loss: 532.247
[1/23] [281/375] loss: 534.156
[1/23] [282/375] loss: 536.120
[1/23] [283/375] loss: 537.971
[1/23] [284/375] loss: 539.520
[1/23] [285/375] loss: 541.293
[1/23] [286/375] loss: 543.016
[1/23] [287/375] loss: 544.986
[1/23] [288/375] loss: 546.535
[1/23] [289/375] loss: 548.038
[1/23] [290/375] loss: 549.971
[1/23] [291/375] loss: 551.770
[1/23] [292/375] loss: 553.715
[1/23] [293/375] loss: 555.262
[1/23] [294/375] loss: 556.747
[1/23] [295/375] loss: 558.371
[1/23] [296/375] loss: 559.661
[1/23] [297/375] loss: 561.169
[1/23] [298/375] loss: 562.736
[1/23] [299/375] loss: 564.616
[1/23] [300/375] loss: 566.536
[1/23] [301/375] loss: 568.294
[1/23] [

[2/23] [165/375] loss: 275.309
[2/23] [166/375] loss: 277.168
[2/23] [167/375] loss: 278.939
[2/23] [168/375] loss: 280.512
[2/23] [169/375] loss: 282.021
[2/23] [170/375] loss: 283.630
[2/23] [171/375] loss: 285.418
[2/23] [172/375] loss: 287.309
[2/23] [173/375] loss: 289.089
[2/23] [174/375] loss: 290.913
[2/23] [175/375] loss: 292.208
[2/23] [176/375] loss: 294.120
[2/23] [177/375] loss: 296.189
[2/23] [178/375] loss: 297.848
[2/23] [179/375] loss: 299.376
[2/23] [180/375] loss: 301.119
[2/23] [181/375] loss: 302.671
[2/23] [182/375] loss: 304.297
[2/23] [183/375] loss: 305.839
[2/23] [184/375] loss: 307.452
[2/23] [185/375] loss: 308.832
[2/23] [186/375] loss: 310.373
[2/23] [187/375] loss: 311.841
[2/23] [188/375] loss: 313.671
[2/23] [189/375] loss: 315.483
[2/23] [190/375] loss: 317.032
[2/23] [191/375] loss: 318.845
[2/23] [192/375] loss: 320.256
[2/23] [193/375] loss: 322.006
[2/23] [194/375] loss: 324.074
[2/23] [195/375] loss: 325.794
[2/23] [196/375] loss: 327.590
[2/23] [

[3/23] [58/375] loss: 93.172
[3/23] [59/375] loss: 94.949
[3/23] [60/375] loss: 96.755
[3/23] [61/375] loss: 97.755
[3/23] [62/375] loss: 99.313
[3/23] [63/375] loss: 100.659
[3/23] [64/375] loss: 102.477
[3/23] [65/375] loss: 104.180
[3/23] [66/375] loss: 105.695
[3/23] [67/375] loss: 107.406
[3/23] [68/375] loss: 108.698
[3/23] [69/375] loss: 110.320
[3/23] [70/375] loss: 111.901
[3/23] [71/375] loss: 113.511
[3/23] [72/375] loss: 115.414
[3/23] [73/375] loss: 116.920
[3/23] [74/375] loss: 118.354
[3/23] [75/375] loss: 120.023
[3/23] [76/375] loss: 121.280
[3/23] [77/375] loss: 122.993
[3/23] [78/375] loss: 124.665
[3/23] [79/375] loss: 126.481
[3/23] [80/375] loss: 127.857
[3/23] [81/375] loss: 129.628
[3/23] [82/375] loss: 130.815
[3/23] [83/375] loss: 132.533
[3/23] [84/375] loss: 134.231
[3/23] [85/375] loss: 136.000
[3/23] [86/375] loss: 137.731
[3/23] [87/375] loss: 139.835
[3/23] [88/375] loss: 141.433
[3/23] [89/375] loss: 142.825
[3/23] [90/375] loss: 144.162
[3/23] [91/375]

[3/23] [324/375] loss: 512.629
[3/23] [325/375] loss: 513.918
[3/23] [326/375] loss: 515.603
[3/23] [327/375] loss: 517.135
[3/23] [328/375] loss: 518.826
[3/23] [329/375] loss: 520.342
[3/23] [330/375] loss: 521.828
[3/23] [331/375] loss: 523.433
[3/23] [332/375] loss: 525.122
[3/23] [333/375] loss: 526.674
[3/23] [334/375] loss: 528.329
[3/23] [335/375] loss: 530.041
[3/23] [336/375] loss: 531.573
[3/23] [337/375] loss: 533.537
[3/23] [338/375] loss: 534.621
[3/23] [339/375] loss: 536.084
[3/23] [340/375] loss: 537.301
[3/23] [341/375] loss: 538.945
[3/23] [342/375] loss: 540.633
[3/23] [343/375] loss: 541.970
[3/23] [344/375] loss: 543.294
[3/23] [345/375] loss: 544.554
[3/23] [346/375] loss: 545.762
[3/23] [347/375] loss: 547.177
[3/23] [348/375] loss: 548.447
[3/23] [349/375] loss: 550.121
[3/23] [350/375] loss: 552.197
[3/23] [351/375] loss: 553.927
[3/23] [352/375] loss: 555.722
[3/23] [353/375] loss: 557.540
[3/23] [354/375] loss: 558.997
[3/23] [355/375] loss: 560.312
[3/23] [

...

[20/23] [321/375] loss: 88.945
[20/23] [322/375] loss: 89.508
[20/23] [323/375] loss: 90.027
[20/23] [324/375] loss: 90.513
[20/23] [325/375] loss: 90.608
[20/23] [326/375] loss: 91.255
[20/23] [327/375] loss: 91.945
[20/23] [328/375] loss: 92.128
[20/23] [329/375] loss: 92.187
[20/23] [330/375] loss: 92.571
[20/23] [331/375] loss: 92.860
[20/23] [332/375] loss: 93.207
[20/23] [333/375] loss: 93.338
[20/23] [334/375] loss: 93.422
[20/23] [335/375] loss: 94.177
[20/23] [336/375] loss: 94.566
[20/23] [337/375] loss: 94.712
[20/23] [338/375] loss: 95.049
[20/23] [339/375] loss: 95.344
[20/23] [340/375] loss: 95.601
[20/23] [341/375] loss: 95.914
[20/23] [342/375] loss: 96.103
[20/23] [343/375] loss: 96.534
[20/23] [344/375] loss: 96.664
[20/23] [345/375] loss: 96.816
[20/23] [346/375] loss: 97.012
[20/23] [347/375] loss: 97.355
[20/23] [348/375] loss: 97.859
[20/23] [349/375] loss: 97.990
[20/23] [350/375] loss: 98.097
[20/23] [351/375] loss: 98.531
[20/23] [352/375] loss: 99.060
[20/23] 

[21/23] [216/375] loss: 44.709
[21/23] [217/375] loss: 44.783
[21/23] [218/375] loss: 45.010
[21/23] [219/375] loss: 45.146
[21/23] [220/375] loss: 45.492
[21/23] [221/375] loss: 45.820
[21/23] [222/375] loss: 46.169
[21/23] [223/375] loss: 46.432
[21/23] [224/375] loss: 46.540
[21/23] [225/375] loss: 46.606
[21/23] [226/375] loss: 46.870
[21/23] [227/375] loss: 47.041
[21/23] [228/375] loss: 47.420
[21/23] [229/375] loss: 47.809
[21/23] [230/375] loss: 48.022
[21/23] [231/375] loss: 48.082
[21/23] [232/375] loss: 48.437
[21/23] [233/375] loss: 49.115
[21/23] [234/375] loss: 49.215
[21/23] [235/375] loss: 49.481
[21/23] [236/375] loss: 49.785
[21/23] [237/375] loss: 49.966
[21/23] [238/375] loss: 50.239
[21/23] [239/375] loss: 50.351
[21/23] [240/375] loss: 50.395
[21/23] [241/375] loss: 50.593
[21/23] [242/375] loss: 50.771
[21/23] [243/375] loss: 50.939
[21/23] [244/375] loss: 51.132
[21/23] [245/375] loss: 51.195
[21/23] [246/375] loss: 51.363
[21/23] [247/375] loss: 51.488
[21/23] 

[22/23] [110/375] loss: 18.471
[22/23] [111/375] loss: 18.540
[22/23] [112/375] loss: 18.622
[22/23] [113/375] loss: 18.731
[22/23] [114/375] loss: 18.866
[22/23] [115/375] loss: 19.096
[22/23] [116/375] loss: 19.444
[22/23] [117/375] loss: 19.515
[22/23] [118/375] loss: 19.683
[22/23] [119/375] loss: 19.786
[22/23] [120/375] loss: 19.827
[22/23] [121/375] loss: 19.883
[22/23] [122/375] loss: 19.939
[22/23] [123/375] loss: 19.981
[22/23] [124/375] loss: 20.040
[22/23] [125/375] loss: 20.048
[22/23] [126/375] loss: 20.315
[22/23] [127/375] loss: 20.322
[22/23] [128/375] loss: 20.377
[22/23] [129/375] loss: 20.597
[22/23] [130/375] loss: 20.706
[22/23] [131/375] loss: 20.721
[22/23] [132/375] loss: 20.807
[22/23] [133/375] loss: 20.852
[22/23] [134/375] loss: 20.920
[22/23] [135/375] loss: 20.959
[22/23] [136/375] loss: 21.045
[22/23] [137/375] loss: 21.205
[22/23] [138/375] loss: 21.238
[22/23] [139/375] loss: 21.699
[22/23] [140/375] loss: 21.750
[22/23] [141/375] loss: 21.788
[22/23] 

[23/23] [1/375] loss: 0.241
[23/23] [2/375] loss: 0.458
[23/23] [3/375] loss: 0.535
[23/23] [4/375] loss: 0.690
[23/23] [5/375] loss: 1.078
[23/23] [6/375] loss: 1.147
[23/23] [7/375] loss: 1.474
[23/23] [8/375] loss: 1.495
[23/23] [9/375] loss: 1.595
[23/23] [10/375] loss: 1.610
[23/23] [11/375] loss: 1.893
[23/23] [12/375] loss: 1.932
[23/23] [13/375] loss: 1.965
[23/23] [14/375] loss: 2.112
[23/23] [15/375] loss: 2.157
[23/23] [16/375] loss: 2.216
[23/23] [17/375] loss: 2.556
[23/23] [18/375] loss: 2.632
[23/23] [19/375] loss: 2.670
[23/23] [20/375] loss: 2.808
[23/23] [21/375] loss: 2.957
[23/23] [22/375] loss: 2.992
[23/23] [23/375] loss: 3.175
[23/23] [24/375] loss: 3.200
[23/23] [25/375] loss: 3.377
[23/23] [26/375] loss: 3.473
[23/23] [27/375] loss: 3.577
[23/23] [28/375] loss: 3.847
[23/23] [29/375] loss: 3.898
[23/23] [30/375] loss: 4.052
[23/23] [31/375] loss: 4.185
[23/23] [32/375] loss: 4.432
[23/23] [33/375] loss: 4.582
[23/23] [34/375] loss: 4.606
[23/23] [35/375] loss: 

[23/23] [271/375] loss: 47.362
[23/23] [272/375] loss: 47.729
[23/23] [273/375] loss: 47.900
[23/23] [274/375] loss: 48.114
[23/23] [275/375] loss: 48.356
[23/23] [276/375] loss: 48.404
[23/23] [277/375] loss: 49.154
[23/23] [278/375] loss: 49.684
[23/23] [279/375] loss: 49.897
[23/23] [280/375] loss: 50.093
[23/23] [281/375] loss: 50.125
[23/23] [282/375] loss: 50.189
[23/23] [283/375] loss: 50.853
[23/23] [284/375] loss: 51.241
[23/23] [285/375] loss: 51.992
[23/23] [286/375] loss: 52.600
[23/23] [287/375] loss: 53.220
[23/23] [288/375] loss: 53.469
[23/23] [289/375] loss: 53.681
[23/23] [290/375] loss: 53.896
[23/23] [291/375] loss: 54.064
[23/23] [292/375] loss: 54.154
[23/23] [293/375] loss: 54.300
[23/23] [294/375] loss: 54.326
[23/23] [295/375] loss: 54.484
[23/23] [296/375] loss: 54.662
[23/23] [297/375] loss: 54.787
[23/23] [298/375] loss: 55.103
[23/23] [299/375] loss: 55.450
[23/23] [300/375] loss: 55.586
[23/23] [301/375] loss: 55.851
[23/23] [302/375] loss: 55.992
[23/23] 

In [52]:
def test(model,dataloader):
    res=[]
    for data in dataloader:
        data=data.to(device)
        outputs=model(data)
        _, predict = torch.max(outputs.data, 1)
        predict=predict.tolist()
        res+=predict
    return res

testset=ICHDataset('TestingData',train=False)
dataloader=DataLoader(testset,batch_size,shuffle=False)
res=test(resnet18,dataloader)
print(res)

[1, 4, 5, 1, 2, 0, 2, 1, 4, 5, 0, 2, 2, 3, 0, 1, 3, 4, 5, 4, 0, 2, 4, 3, 1, 0, 2, 0, 3, 4, 3, 1, 2, 2, 2, 0, 3, 5, 2, 5, 5, 5, 4, 1, 0, 1, 0, 1, 3, 5, 2, 5, 0, 0, 0, 1, 4, 0, 1, 5, 3, 1, 1, 3, 5, 4, 3, 2, 1, 3, 0, 2, 5, 3, 5, 3, 5, 0, 0, 1, 4, 0, 2, 2, 2, 2, 1, 3, 0, 3, 5, 1, 1, 1, 4, 2, 1, 5, 1, 3, 2, 1, 0, 4, 1, 3, 4, 4, 1, 3, 1, 2, 1, 0, 0, 5, 2, 1, 3, 0, 0, 3, 2, 4, 5, 5, 2, 1, 2, 3, 5, 5, 0, 2, 5, 4, 0, 0, 1, 3, 3, 2, 1, 0, 5, 4, 5, 3, 3, 0, 2, 3, 1, 1, 0, 1, 1, 2, 5, 3, 2, 2, 4, 2, 0, 4, 3, 5, 0, 5, 2, 4, 5, 3, 4, 0, 2, 0, 3, 1, 0, 4, 3, 1, 1, 5, 0, 2, 2, 4, 1, 2, 3, 2, 4, 1, 1, 3, 0, 2, 5, 2, 1, 4, 2, 1, 4, 2, 4, 4, 3, 1, 2, 0, 1, 3, 1, 5, 0, 1, 2, 5, 4, 3, 2, 1, 5, 3, 2, 3, 0, 0, 2, 1, 0, 4, 1, 3, 3, 3, 1, 1, 5, 3, 2, 4, 1, 0, 2, 4, 4, 2, 3, 4, 2, 2, 3, 4, 0, 5, 3, 1, 1, 2, 5, 4, 1, 2, 0, 2, 3, 5, 4, 2, 4, 1, 1, 3, 1, 5, 1, 3, 0, 2, 0, 3, 1, 1, 0, 2, 1, 5, 5, 4, 3, 4, 1, 1, 4, 0, 2, 4, 2, 3, 1, 1, 0, 5, 4, 5, 3, 0, 1, 4, 0, 2, 2, 2, 5, 1, 1, 0, 2, 1, 4, 0, 0, 4, 3, 5, 0, 2, 3, 

In [53]:
# write excel file
wb=Workbook()
ws=wb.active
for i in range(len(testset)):
    val=testset.filename_list[i].split('/')[1]
    val=val.split('.')[0]
    ws.cell(i+1,1).value=val
    ws.cell(i+1,2).value=class_dict[res[i]]
wb.save('testing_result.xlsx')